# Code samples -- Data Preparation

by João Oda

The main goal of this notebook is to exemplify part of the data preparation process of a data science project. 

I will go through the preparation steps and later, summary the process with a diagram and add some comments.

## The context fo this project

The SAT is a standardized test widely used for college admissions in the United States.
We want to study demographic factors that relates with SAT score performance of schools. The data comes from multiple sources in multiple formats and need pre-processing before analysis.


### Some domain knowledge about our data

We are going to use New York City data on high school SAT scores available online.

New York City is made up of five boroughs, which are essentially distinct regions.

New York City schools fall within several different school districts, each of which can contains dozens of schools.

Each school in New York City has a unique code called a DBN, or district borough number.

Aggregating data by district will allow us to use the district mapping data to plot district-by-district differences.

## Data source

In this project our data comes from multiples files, in multiple formats. Lets read them:

In [1]:
import pandas as pd
import numpy as np
import re
from bokeh.io import output_notebook, show
from bokeh.plotting import figure

output_notebook()

Loading BokehJS ...

In [2]:
data = {}
data['ap_2010'] = pd.read_csv('data/NY_schools/raw/AP__College_Board__2010_School_Level_Results.csv')
data['class_size'] = pd.read_csv('data/NY_schools/raw/2010-2011_Class_Size_-_School-level_detail.csv')
data['demographics'] = pd.read_csv('data/NY_schools/raw/School_Demographics_and_Accountability_Snapshot_2006-2012.csv')
data['graduation'] = pd.read_csv('data/NY_schools/raw/Graduation_Outcomes_-_Classes_Of_2005-2010_-_School_Level.csv')
data['hs_directory'] = pd.read_csv('data/NY_schools/raw/DOE_High_School_Directory_2014-2015.csv') 
data['sat_results'] = pd.read_csv('data/NY_schools/raw/SAT_Results.csv')

In [3]:
txt_files = [
    'masterfile11_gened_final.txt',
    'masterfile11_d75_final.txt'
]
surveys_dfs = [pd.read_csv('data/NY_schools/raw/{0}'.format(f),
                           delimiter='\t',
                           encoding='windows-1252') for f in txt_files]

We are going to take a closer look at each dataframe and prepare them to be merged into a single dataframe.

### Surveys

Surveys of parents, teachers, and students at each school

In [4]:
print(surveys_dfs[0].shape)
surveys_dfs[0].head()

(1646, 1942)


,dbn,bn,schoolname,d75,studentssurveyed,highschool,schooltype,rr_s,rr_t,rr_p,...,s_N_q14e_3,s_N_q14e_4,s_N_q14f_1,s_N_q14f_2,s_N_q14f_3,s_N_q14f_4,s_N_q14g_1,s_N_q14g_2,s_N_q14g_3,s_N_q14g_4
0,01M015,M015,P.S. 015 Roberto Clemente,0,No,0.0,Elementary School,NaN,88,60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01M019,M019,P.S. 019 Asher Levy,0,No,0.0,Elementary School,NaN,100,60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01M020,M020,P.S. 020 Anna Silver,0,No,0.0,Elementary School,NaN,88,73,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01M034,M034,P.S. 034 Franklin D. Roosevelt,0,Yes,0.0,Elementary / Middle School,89.0,73,50,...,20.0,16.0,23.0,54.0,33.0,29.0,31.0,46.0,16.0,8.0
4,01M063,M063,P.S. 063 William McKinley,0,No,0.0,Elementary School,NaN,100,60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
print(surveys_dfs[1].shape)
surveys_dfs[1].head()

(56, 1773)


,dbn,bn,schoolname,d75,studentssurveyed,highschool,schooltype,rr_s,rr_t,rr_p,...,s_q14_2,s_q14_3,s_q14_4,s_q14_5,s_q14_6,s_q14_7,s_q14_8,s_q14_9,s_q14_10,s_q14_11
0,75K004,K004,P.S. K004,1,Yes,0.0,District 75 Special Education,38.0,90,72,...,29.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,75K036,K036,P.S. 36,1,Yes,NaN,District 75 Special Education,70.0,69,44,...,20.0,27.0,19.0,9.0,2.0,6.0,1.0,2.0,0.0,0.0
2,75K053,K053,P.S. K053,1,Yes,NaN,District 75 Special Education,94.0,97,53,...,14.0,12.0,12.0,10.0,21.0,13.0,11.0,2.0,0.0,0.0
3,75K077,K077,P.S. K077,1,Yes,NaN,District 75 Special Education,95.0,65,55,...,14.0,14.0,7.0,11.0,16.0,10.0,6.0,4.0,7.0,7.0
4,75K140,K140,P.S. K140,1,Yes,0.0,District 75 Special Education,77.0,70,42,...,35.0,34.0,17.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0


The survey data is split in two dataframes, although they have different number of columns, they have columns in common. We combine both and get a dataframe where the columns are the union os the previous set of columns.

In [6]:
survey = pd.concat(surveys_dfs, axis=0)
print(survey.shape)
survey.head()

(1702, 2773)


,N_p,N_s,N_t,aca_p_11,aca_s_11,aca_t_11,aca_tot_11,bn,com_p_11,com_s_11,...,t_q8c_1,t_q8c_2,t_q8c_3,t_q8c_4,t_q9,t_q9_1,t_q9_2,t_q9_3,t_q9_4,t_q9_5
0,90.0,NaN,22.0,7.8,NaN,7.9,7.9,M015,7.6,NaN,...,29.0,67.0,5.0,0.0,NaN,5.0,14.0,52.0,24.0,5.0
1,161.0,NaN,34.0,7.8,NaN,9.1,8.4,M019,7.6,NaN,...,74.0,21.0,6.0,0.0,NaN,3.0,6.0,3.0,78.0,9.0
2,367.0,NaN,42.0,8.6,NaN,7.5,8.0,M020,8.3,NaN,...,33.0,35.0,20.0,13.0,NaN,3.0,5.0,16.0,70.0,5.0
3,151.0,145.0,29.0,8.5,7.4,7.8,7.9,M034,8.2,5.9,...,21.0,45.0,28.0,7.0,NaN,0.0,18.0,32.0,39.0,11.0
4,90.0,NaN,23.0,7.9,NaN,8.1,8.0,M063,7.9,NaN,...,59.0,36.0,5.0,0.0,NaN,10.0,5.0,10.0,60.0,15.0


Based on domain knowledge lets reduces the numbers of columns we are going to look at:

| Field         | Description                                           |
| ------------- |:-------------:                                        |
| dbn           | School identification code (district borough number)  |
| rr_s          | Student Response Rate |
| rr_t          | Teacher Response Rate |
| rr_p          | Parent Response Rate  |
| N_s           | Number of student respondents |
| N_s           | Number of student respondents |
| N_t           | Number of student respondents |
| N_p           | Number of student respondents |
|com_p_11	|Communication score based on parent responses |
|eng_p_11	|Engagement score based on parent responses |
|aca_p_11	|Academic expectations score based on parent responses |
|saf_t_11	|Safety and Respect score based on teacher responses |
|com_t_11	|Communication score based on teacher responses |
|eng_t_11	|Engagement score based on teacher responses |
|aca_t_11	|Academic expectations score based on teacher responses |
|saf_s_11	|Safety and Respect score based on student responses |
|com_s_11	|Communication score based on student responses |
|eng_s_11	|Engagement score based on student responses |
|aca_s_11	|Academic expectations score based on student responses |
|saf_tot_11	|Safety and Respect total score |
|com_tot_11	|Communication total score |
|eng_tot_11	|Engagement total score |
|aca_tot_11	|Academic Expectations total score |

In [7]:
survey['DBN'] = survey['dbn'] #for consistent with the other data sets

In [8]:
survey_fields = [
    'DBN', 
    'rr_s', 
    'rr_t', 
    'rr_p', 
    'N_s', 
    'N_t', 
    'N_p', 
    'saf_p_11', 
    'com_p_11', 
    'eng_p_11', 
    'aca_p_11', 
    'saf_t_11', 
    'com_t_11', 
    'eng_t_11', 
    'aca_t_11', 
    'saf_s_11', 
    'com_s_11', 
    'eng_s_11', 
    'aca_s_11', 
    'saf_tot_11', 
    'com_tot_11', 
    'eng_tot_11', 
    'aca_tot_11',
]

survey = survey.loc[:,survey_fields]
data['survey'] = survey
data['survey'].head()

,DBN,rr_s,rr_t,rr_p,N_s,N_t,N_p,saf_p_11,com_p_11,eng_p_11,...,eng_t_11,aca_t_11,saf_s_11,com_s_11,eng_s_11,aca_s_11,saf_tot_11,com_tot_11,eng_tot_11,aca_tot_11
0,01M015,NaN,88,60,NaN,22.0,90.0,8.5,7.6,7.5,...,7.6,7.9,NaN,NaN,NaN,NaN,8.0,7.7,7.5,7.9
1,01M019,NaN,100,60,NaN,34.0,161.0,8.4,7.6,7.6,...,8.9,9.1,NaN,NaN,NaN,NaN,8.5,8.1,8.2,8.4
2,01M020,NaN,88,73,NaN,42.0,367.0,8.9,8.3,8.3,...,6.8,7.5,NaN,NaN,NaN,NaN,8.2,7.3,7.5,8.0
3,01M034,89.0,73,50,145.0,29.0,151.0,8.8,8.2,8.0,...,6.8,7.8,6.2,5.9,6.5,7.4,7.3,6.7,7.1,7.9
4,01M063,NaN,100,60,NaN,23.0,90.0,8.7,7.9,8.1,...,7.8,8.1,NaN,NaN,NaN,NaN,8.5,7.6,7.9,8.0


In [9]:
data['survey'].duplicated(subset=['DBN']).sum()

0

## Class_size

Average class sizes for each school, by grade and program type (General Education, Self-Contained Special Education, Collaborative Team Teaching (CTT)) for grades K-9 (where grade 9 is not reported by subject area), and for grades 5-9 (where available) and 9-12, aggregated by program type (General Education, CTT, and Self-Contained Special Education) and core course (e.g. English 9, Integrated Algebra, US History, etc.).

Class size data is based on January 28, 2011 data. 

source: https://data.cityofnewyork.us/Education/2010-2011-Class-Size-School-level-detail/urz7-pzb3

In [10]:
print(data['class_size'].shape)
data['class_size'].head()

(27611, 16)


,CSD,BOROUGH,SCHOOL CODE,SCHOOL NAME,GRADE,PROGRAM TYPE,CORE SUBJECT (MS CORE and 9-12 ONLY),CORE COURSE (MS CORE and 9-12 ONLY),SERVICE CATEGORY(K-9* ONLY),NUMBER OF STUDENTS / SEATS FILLED,NUMBER OF SECTIONS,AVERAGE CLASS SIZE,SIZE OF SMALLEST CLASS,SIZE OF LARGEST CLASS,DATA SOURCE,SCHOOLWIDE PUPIL-TEACHER RATIO
0,1,M,M015,P.S. 015 Roberto Clemente,0K,GEN ED,-,-,-,19.0,1.0,19.0,19.0,19.0,ATS,NaN
1,1,M,M015,P.S. 015 Roberto Clemente,0K,CTT,-,-,-,21.0,1.0,21.0,21.0,21.0,ATS,NaN
2,1,M,M015,P.S. 015 Roberto Clemente,01,GEN ED,-,-,-,17.0,1.0,17.0,17.0,17.0,ATS,NaN
3,1,M,M015,P.S. 015 Roberto Clemente,01,CTT,-,-,-,17.0,1.0,17.0,17.0,17.0,ATS,NaN
4,1,M,M015,P.S. 015 Roberto Clemente,02,GEN ED,-,-,-,15.0,1.0,15.0,15.0,15.0,ATS,NaN


In future we are goint to use the DBN to associate data over the dataframes, althought we miss it here, we can generate it from other columns. 

First let's check the consistency of the columns of that will generate the DBN:

In [11]:
data['class_size']['CSD'].unique()

array([ 1,  2, 21, 27,  6,  3,  5,  7,  4,  8,  9, 10, 11, 12, 13, 14, 15,
       16, 17, 19, 18, 20, 22, 23, 24, 25, 26, 28, 29, 30, 31, 32])

In [12]:
data['class_size']['SCHOOL CODE'].apply(
    lambda x :re.search('^[A-Z][0-9][0-9]', x) is not None).describe()

count     27611
unique        1
top        True
freq      27611
Name: SCHOOL CODE, dtype: object

The formart of SCHOOL CODE seens to be ok and we just need to pad CSD then we can generate DBN

In [13]:
def pad_csd(num):
    string_representation = str(num)
    if len(string_representation) > 1:
        return string_representation
    else:
        return string_representation.zfill(2)


data['class_size']['padded_csd'] = data['class_size']['CSD'].apply(pad_csd)
data['class_size']['DBN'] = data['class_size']['padded_csd'] + data['class_size']['SCHOOL CODE']

In [14]:
data["class_size"].head()

,CSD,BOROUGH,SCHOOL CODE,SCHOOL NAME,GRADE,PROGRAM TYPE,CORE SUBJECT (MS CORE and 9-12 ONLY),CORE COURSE (MS CORE and 9-12 ONLY),SERVICE CATEGORY(K-9* ONLY),NUMBER OF STUDENTS / SEATS FILLED,NUMBER OF SECTIONS,AVERAGE CLASS SIZE,SIZE OF SMALLEST CLASS,SIZE OF LARGEST CLASS,DATA SOURCE,SCHOOLWIDE PUPIL-TEACHER RATIO,padded_csd,DBN
0,1,M,M015,P.S. 015 Roberto Clemente,0K,GEN ED,-,-,-,19.0,1.0,19.0,19.0,19.0,ATS,NaN,01,01M015
1,1,M,M015,P.S. 015 Roberto Clemente,0K,CTT,-,-,-,21.0,1.0,21.0,21.0,21.0,ATS,NaN,01,01M015
2,1,M,M015,P.S. 015 Roberto Clemente,01,GEN ED,-,-,-,17.0,1.0,17.0,17.0,17.0,ATS,NaN,01,01M015
3,1,M,M015,P.S. 015 Roberto Clemente,01,CTT,-,-,-,17.0,1.0,17.0,17.0,17.0,ATS,NaN,01,01M015
4,1,M,M015,P.S. 015 Roberto Clemente,02,GEN ED,-,-,-,15.0,1.0,15.0,15.0,15.0,ATS,NaN,01,01M015


In [15]:
data['class_size']['DBN'].nunique()

1487

We don't have unique values for DBN. That's due multiple values for
GRADES and PROGRAM TYPE.

In [16]:
data['class_size']['GRADE '].unique()

array(['0K', '01', '02', '03', '04', '05', '0K-09', nan, '06', '07', '08',
       'MS Core', '09-12', '09'], dtype=object)

Because we're dealing with high schools, we're only concerned with grades 9 through 12

In [17]:
class_size = data['class_size']
class_size = class_size[class_size['GRADE '] == '09-12']

Let's check PROGRAM TYPE now:

In [18]:
class_size.groupby("PROGRAM TYPE")['DBN'].nunique()

PROGRAM TYPE
CTT        400
GEN ED     583
SPEC ED    214
Name: DBN, dtype: int64

We are going to take only largest category GEN ED:

In [19]:
class_size = class_size[class_size["PROGRAM TYPE"] == "GEN ED"]

In [20]:
class_size.duplicated(subset=["DBN"]).sum()

5930

We don't have unique values for DBN. Lets check a particular DBN:

In [21]:
class_size[class_size['DBN']== '01M292']

,CSD,BOROUGH,SCHOOL CODE,SCHOOL NAME,GRADE,PROGRAM TYPE,CORE SUBJECT (MS CORE and 9-12 ONLY),CORE COURSE (MS CORE and 9-12 ONLY),SERVICE CATEGORY(K-9* ONLY),NUMBER OF STUDENTS / SEATS FILLED,NUMBER OF SECTIONS,AVERAGE CLASS SIZE,SIZE OF SMALLEST CLASS,SIZE OF LARGEST CLASS,DATA SOURCE,SCHOOLWIDE PUPIL-TEACHER RATIO,padded_csd,DBN
225,1,M,M292,Henry Street School for International Studies,09-12,GEN ED,ENGLISH,English 9,-,63.0,3.0,21.0,19.0,25.0,STARS,NaN,01,01M292
226,1,M,M292,Henry Street School for International Studies,09-12,GEN ED,ENGLISH,English 10,-,79.0,3.0,26.3,24.0,31.0,STARS,NaN,01,01M292
227,1,M,M292,Henry Street School for International Studies,09-12,GEN ED,ENGLISH,English 11,-,38.0,2.0,19.0,16.0,22.0,STARS,NaN,01,01M292
228,1,M,M292,Henry Street School for International Studies,09-12,GEN ED,ENGLISH,English 12,-,69.0,3.0,23.0,13.0,30.0,STARS,NaN,01,01M292
229,1,M,M292,Henry Street School for International Studies,09-12,GEN ED,MATH,Integrated Algebra,-,53.0,3.0,17.7,16.0,21.0,STARS,NaN,01,01M292
231,1,M,M292,Henry Street School for International Studies,09-12,GEN ED,MATH,Geometry,-,32.0,1.0,32.0,32.0,32.0,STARS,NaN,01,01M292
232,1,M,M292,Henry Street School for International Studies,09-12,GEN ED,MATH,Other Math,-,118.0,6.0,19.7,13.0,27.0,STARS,NaN,01,01M292
233,1,M,M292,Henry Street School for International Studies,09-12,GEN ED,SCIENCE,Earth Science,-,125.0,4.0,31.3,28.0,35.0,STARS,NaN,01,01M292
234,1,M,M292,Henry Street School for International Studies,09-12,GEN ED,SCIENCE,Living Environment,-,58.0,2.0,29.0,29.0,29.0,STARS,NaN,01,01M292
235,1,M,M292,Henry Street School for International Studies,09-12,GEN ED,SCIENCE,Chemistry,-,157.0,8.0,19.6,13.0,24.0,STARS,NaN,01,01M292


The diference comes from multiple subjects and course. We are going to agregate the values and take the mean

In [22]:
class_size = class_size.groupby('DBN').agg(np.mean)
class_size.reset_index(inplace=True)

In [23]:
class_size.duplicated(subset=['DBN']).sum()

0

In [24]:
data['class_size'] = class_size
data['class_size'].head()

,DBN,CSD,NUMBER OF STUDENTS / SEATS FILLED,NUMBER OF SECTIONS,AVERAGE CLASS SIZE,SIZE OF SMALLEST CLASS,SIZE OF LARGEST CLASS,SCHOOLWIDE PUPIL-TEACHER RATIO
0,01M292,1,88.0000,4.000000,22.564286,18.50,26.571429,NaN
1,01M332,1,46.0000,2.000000,22.000000,21.00,23.500000,NaN
2,01M378,1,33.0000,1.000000,33.000000,33.00,33.000000,NaN
3,01M448,1,105.6875,4.750000,22.231250,18.25,27.062500,NaN
4,01M450,1,57.6000,2.733333,21.200000,19.40,22.866667,NaN


### Directory of high schools data

Directory of NYC High Schools. We assume invariance in time from 2011 and 2014.

from https://data.cityofnewyork.us/Education/2014-2015-DOE-High-School-Directory/n3p6-zve2

In [25]:
data['hs_directory'].head()

,dbn,school_name,borough,building_code,phone_number,fax_number,grade_span_min,grade_span_max,expgrade_span_min,expgrade_span_max,...,priority08,priority09,priority10,Location 1,Community Board,Council District,Census Tract,BIN,BBL,NTA
0,17K548,Brooklyn School for Music & Theatre,Brooklyn,K440,718-230-6250,718-230-6262,9.0,12,NaN,NaN,...,NaN,NaN,NaN,"883 Classon Avenue\nBrooklyn, NY 11225\n(40.67...",9.0,35.0,213.0,3029686.0,3.011870e+09,Crown Heights South ...
1,09X543,High School for Violin and Dance,Bronx,X400,718-842-0687,718-589-9849,9.0,12,NaN,NaN,...,NaN,NaN,NaN,"1110 Boston Road\nBronx, NY 10456\n(40.8276026...",3.0,16.0,135.0,2004526.0,2.026340e+09,Morrisania-Melrose ...
2,09X327,Comprehensive Model School Project M.S. 327,Bronx,X240,718-294-8111,718-294-8109,6.0,12,NaN,NaN,...,NaN,NaN,NaN,"1501 Jerome Avenue\nBronx, NY 10452\n(40.84241...",4.0,14.0,209.0,2008336.0,2.028590e+09,West Concourse ...
3,02M280,Manhattan Early College School for Advertising,Manhattan,M520,718-935-3477,NaN,9.0,10,9.0,14.0,...,NaN,NaN,NaN,"411 Pearl Street\nNew York, NY 10038\n(40.7106...",1.0,1.0,29.0,1001388.0,1.001130e+09,Chinatown ...
4,28Q680,Queens Gateway to Health Sciences Secondary Sc...,Queens,Q695,718-969-3155,718-969-3552,6.0,12,NaN,NaN,...,NaN,NaN,NaN,"160 20 Goethals Avenue\nJamaica, NY 11432\n(40...",8.0,24.0,1267.0,4539721.0,4.068580e+09,Pomonok-Flushing Heights-Hillcrest ...


In [26]:
data['hs_directory']['DBN'] = data['hs_directory']['dbn']

We are going to parse lat and long from "Location 1" column then convert to numeric. 

In [27]:
def find_lat(loc):
    coords = re.findall('\(.+\)', loc)
    lat = coords[0].split(',')[0].replace('(', '')
    return lat

def find_lon(loc):
    coords = re.findall('\(.+\)', loc)
    lon = coords[0].split(',')[1].replace(')', '').strip()
    return lon


data['hs_directory']['lat'] = data['hs_directory']['Location 1'].apply(find_lat)

data['hs_directory']['lon'] = data['hs_directory']['Location 1'].apply(find_lon)

data['hs_directory']['lat'] = pd.to_numeric(data['hs_directory']['lat'], errors='coerce') # coerse optiton - invalid
data['hs_directory']['lon'] = pd.to_numeric(data['hs_directory']['lon'], errors='coerce') # parsing will be set as NaN


In [28]:
data['hs_directory']['lat'].describe()

count    435.000000
mean      40.745473
std        0.080937
min       40.528229
25%       40.690496
50%       40.741205
75%       40.822096
max       40.888373
Name: lat, dtype: float64

In [29]:
data['hs_directory']['lon'].describe()

count    435.000000
mean     -73.921071
std        0.071705
min      -74.192154
25%      -73.979256
50%      -73.925141
75%      -73.882579
max      -73.726910
Name: lon, dtype: float64

In [30]:
data['hs_directory'].head()

,dbn,school_name,borough,building_code,phone_number,fax_number,grade_span_min,grade_span_max,expgrade_span_min,expgrade_span_max,...,Location 1,Community Board,Council District,Census Tract,BIN,BBL,NTA,DBN,lat,lon
0,17K548,Brooklyn School for Music & Theatre,Brooklyn,K440,718-230-6250,718-230-6262,9.0,12,NaN,NaN,...,"883 Classon Avenue\nBrooklyn, NY 11225\n(40.67...",9.0,35.0,213.0,3029686.0,3.011870e+09,Crown Heights South ...,17K548,40.670299,-73.961648
1,09X543,High School for Violin and Dance,Bronx,X400,718-842-0687,718-589-9849,9.0,12,NaN,NaN,...,"1110 Boston Road\nBronx, NY 10456\n(40.8276026...",3.0,16.0,135.0,2004526.0,2.026340e+09,Morrisania-Melrose ...,09X543,40.827603,-73.904475
2,09X327,Comprehensive Model School Project M.S. 327,Bronx,X240,718-294-8111,718-294-8109,6.0,12,NaN,NaN,...,"1501 Jerome Avenue\nBronx, NY 10452\n(40.84241...",4.0,14.0,209.0,2008336.0,2.028590e+09,West Concourse ...,09X327,40.842414,-73.916162
3,02M280,Manhattan Early College School for Advertising,Manhattan,M520,718-935-3477,NaN,9.0,10,9.0,14.0,...,"411 Pearl Street\nNew York, NY 10038\n(40.7106...",1.0,1.0,29.0,1001388.0,1.001130e+09,Chinatown ...,02M280,40.710679,-74.000807
4,28Q680,Queens Gateway to Health Sciences Secondary Sc...,Queens,Q695,718-969-3155,718-969-3552,6.0,12,NaN,NaN,...,"160 20 Goethals Avenue\nJamaica, NY 11432\n(40...",8.0,24.0,1267.0,4539721.0,4.068580e+09,Pomonok-Flushing Heights-Hillcrest ...,28Q680,40.718810,-73.806500


In [31]:
data['hs_directory']['DBN'].duplicated().sum()

0

### Demographics

Annual school accounts of NYC public school student populations served by grade, special programs, ethnicity, gender and Title I funded programs. 

from https://data.cityofnewyork.us/Education/2006-2012-School-Demographics-and-Accountability-S/ihfw-zy9j

In [32]:
data['demographics']['DBN'].duplicated().sum()

8481

In [33]:
data['demographics'][data['demographics']['DBN']== '01M015']

,DBN,Name,schoolyear,fl_percent,frl_percent,total_enrollment,prek,k,grade1,grade2,...,black_num,black_per,hispanic_num,hispanic_per,white_num,white_per,male_num,male_per,female_num,female_per
0,01M015,P.S. 015 ROBERTO CLEMENTE,20052006,89.4,NaN,281,15,36,40,33,...,74,26.3,189,67.3,5,1.8,158.0,56.2,123.0,43.8
1,01M015,P.S. 015 ROBERTO CLEMENTE,20062007,89.4,NaN,243,15,29,39,38,...,68,28.0,153,63.0,4,1.6,140.0,57.6,103.0,42.4
2,01M015,P.S. 015 ROBERTO CLEMENTE,20072008,89.4,NaN,261,18,43,39,36,...,77,29.5,157,60.2,7,2.7,143.0,54.8,118.0,45.2
3,01M015,P.S. 015 ROBERTO CLEMENTE,20082009,89.4,NaN,252,17,37,44,32,...,75,29.8,149,59.1,7,2.8,149.0,59.1,103.0,40.9
4,01M015,P.S. 015 ROBERTO CLEMENTE,20092010,,96.5,208,16,40,28,32,...,67,32.2,118,56.7,6,2.9,124.0,59.6,84.0,40.4
5,01M015,P.S. 015 ROBERTO CLEMENTE,20102011,,96.5,203,13,37,35,33,...,75,36.9,110,54.2,4,2.0,113.0,55.7,90.0,44.3
6,01M015,P.S. 015 ROBERTO CLEMENTE,20112012,NaN,89.4,189,13,31,35,28,...,63,33.3,109,57.7,4,2.1,97.0,51.3,92.0,48.7


We are only interested in schoolyear 2011-2012

In [34]:
data['demographics'] = data['demographics'][data['demographics']['schoolyear'] == 20112012]

In [35]:
data['demographics'].duplicated(subset=['DBN']).sum()

0

### Graduation

Graduation Outcomes - Cohorts of 2001 through 2006 (Classes of 2005 through 2010) 
The New York State calculation method was first adopted for the Cohort of 2001 (Class of 2005). The cohort consists of all students who first entered 9th grade in a given school year (e.g., the Cohort of 2006 entered 9th grade in the 2006-2007 school year). Graduates are defined as those students earning either a Local or Regents diploma and exclude those earning either a special education (IEP) diploma or GED. For the most recent cohort, graduation rates as of both June and August (including summer graduates) are reported. 
Records with cohorts of 20 students or less are suppressed. August outcomes are only reported for the most recent cohort. 
August outcomes include all June and August graduates. In school-level reporting, students who were in a school for less than 5 months are not included in the school’s cohort, but are included in citywide totals. 
School level results are not presented for District 79 schools, but their outcomes are included in citywide totals. 
Schools are listed by their current DBN's.

from https://data.cityofnewyork.us/Education/2005-2010-Graduation-Outcomes-School-Level/vh2h-md7a

In [36]:
data['graduation'].head()

,Demographic,DBN,School Name,Cohort,Total Cohort,Total Grads - n,Total Grads - % of cohort,Total Regents - n,Total Regents - % of cohort,Total Regents - % of grads,...,Regents w/o Advanced - n,Regents w/o Advanced - % of cohort,Regents w/o Advanced - % of grads,Local - n,Local - % of cohort,Local - % of grads,Still Enrolled - n,Still Enrolled - % of cohort,Dropped Out - n,Dropped Out - % of cohort
0,Total Cohort,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL,2003,5,s,s,s,s,s,...,s,s,s,s,s,s,s,s,s,s
1,Total Cohort,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL,2004,55,37,67.3%,17,30.9%,45.9%,...,17,30.9%,45.9%,20,36.4%,54.1%,15,27.3%,3,5.5%
2,Total Cohort,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL,2005,64,43,67.2%,27,42.2%,62.8%,...,27,42.2%,62.8%,16,25%,37.200000000000003%,9,14.1%,9,14.1%
3,Total Cohort,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL,2006,78,43,55.1%,36,46.2%,83.7%,...,36,46.2%,83.7%,7,9%,16.3%,16,20.5%,11,14.1%
4,Total Cohort,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL,2006 Aug,78,44,56.4%,37,47.4%,84.1%,...,37,47.4%,84.1%,7,9%,15.9%,15,19.2%,11,14.1%


In [37]:
data['graduation']['Demographic'].unique()

array(['Total Cohort', 'Male', 'English Language Learners',
       'English Proficient Students', 'Special Education Students',
       'General Education Students', 'Asian', 'Black', 'Hispanic',
       'White', 'Female'], dtype=object)

In [38]:
data['graduation']['Cohort'].unique()

array(['2003', '2004', '2005', '2006', '2006 Aug', '2001', '2002'],
      dtype=object)

We want to pick data from the most recent Cohort available, which is 2006. We also want data from the full cohort, so we'll only pick rows where Demographic is Total Cohort

In [39]:
data['graduation'] = data['graduation'][data['graduation']['Cohort'] == '2006']
data['graduation'] = data['graduation'][data['graduation']['Demographic'] == 'Total Cohort']

In [40]:
data['graduation']['DBN'].duplicated().sum()

0

### Advanced Placement (AP) test scores

New York City school level College Board AP results for 2010. 

Records with 5 or fewer students are suppressed. 

Students are linked to schools by identifying which school they attend when registering for a College Board exam. A student is only included in a school’s report if he/she self-reports being enrolled at that school. 

Data collected and processed by the College Board.

from https://data.cityofnewyork.us/Education/2010-AP-College-Board-School-Level-Results/itfs-ms3e

AP exams have a 1 to 5 scale; 3 or higher is a passing score. Many high school students take AP exams -- particularly those who attend academically challenging institutions. AP exams are much more rare in schools that lack funding or academic rigor.

In [41]:
data['ap_2010'].head()

,DBN,SchoolName,AP Test Takers,Total Exams Taken,Number of Exams with scores 3 4 or 5
0,01M448,UNIVERSITY NEIGHBORHOOD H.S.,39,49,10
1,01M450,EAST SIDE COMMUNITY HS,19,21,s
2,01M515,LOWER EASTSIDE PREP,24,26,24
3,01M539,"NEW EXPLORATIONS SCI,TECH,MATH",255,377,191
4,02M296,High School of Hospitality Management,s,s,s


It will be interesting to find out whether AP exam scores are correlated with SAT scores across high schools. To determine this, we'll need to convert the AP exam scores in the ap_2010 data set to numeric values first.

In [42]:
data['ap_2010'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 258 entries, 0 to 257
Data columns (total 5 columns):
DBN                                     258 non-null object
SchoolName                              258 non-null object
AP Test Takers                          258 non-null object
Total Exams Taken                       258 non-null object
Number of Exams with scores 3 4 or 5    258 non-null object
dtypes: object(5)
memory usage: 10.2+ KB


In [43]:
cols = [
    'AP Test Takers ',
    'Total Exams Taken',
    'Number of Exams with scores 3 4 or 5'
]

for col in cols:
    data["ap_2010"][col] = pd.to_numeric(data['ap_2010'][col], errors='coerce')

In [44]:
data['ap_2010'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 258 entries, 0 to 257
Data columns (total 5 columns):
DBN                                     258 non-null object
SchoolName                              258 non-null object
AP Test Takers                          233 non-null float64
Total Exams Taken                       233 non-null float64
Number of Exams with scores 3 4 or 5    151 non-null float64
dtypes: float64(3), object(2)
memory usage: 10.2+ KB


In [45]:
data['ap_2010']['DBN'].duplicated().sum()

1

In [46]:
data['ap_2010'][data['ap_2010']['DBN'].duplicated()]

,DBN,SchoolName,AP Test Takers,Total Exams Taken,Number of Exams with scores 3 4 or 5
52,04M610,YOUNG WOMEN'S LEADERSHIP SCH,NaN,NaN,NaN


In [47]:
data['ap_2010'][data['ap_2010']['DBN'] == '04M610']

,DBN,SchoolName,AP Test Takers,Total Exams Taken,Number of Exams with scores 3 4 or 5
51,04M610,THE YOUNG WOMEN'S LEADERSHIP SCHOOL OF EAST HA...,41.0,55.0,29.0
52,04M610,YOUNG WOMEN'S LEADERSHIP SCH,NaN,NaN,NaN


In [48]:
data['ap_2010'] = data['ap_2010'].drop(52)

In [49]:
data['ap_2010'][data['ap_2010']['DBN'] == '04M610']

,DBN,SchoolName,AP Test Takers,Total Exams Taken,Number of Exams with scores 3 4 or 5
51,04M610,THE YOUNG WOMEN'S LEADERSHIP SCHOOL OF EAST HA...,41.0,55.0,29.0


In [50]:
data['ap_2010']['DBN'].duplicated().sum()

0

### Sat Results

The school level results for New York City on the SAT. Results are available at the school level for the graduating seniors of 2012. Records contain 2012 College-bound seniors mean SAT scores taken during SY 2012.

https://data.cityofnewyork.us/Education/2012-SAT-Results/f9bf-2cp4

In [51]:
data['sat_results'].head()

,DBN,SCHOOL NAME,Num of SAT Test Takers,SAT Critical Reading Avg. Score,SAT Math Avg. Score,SAT Writing Avg. Score
0,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,29,355,404,363
1,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,91,383,423,366
2,01M450,EAST SIDE COMMUNITY SCHOOL,70,377,402,370
3,01M458,FORSYTH SATELLITE ACADEMY,7,414,401,359
4,01M509,MARTA VALLE HIGH SCHOOL,44,390,433,384


SAT test consists of three sections, each of which has 800 possible points. The combined score is out of 2,400 possible points (while this number has changed a few times, the data set for our project is based on 2,400 total points)

In [52]:
data['sat_results'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 478 entries, 0 to 477
Data columns (total 6 columns):
DBN                                478 non-null object
SCHOOL NAME                        478 non-null object
Num of SAT Test Takers             478 non-null object
SAT Critical Reading Avg. Score    478 non-null object
SAT Math Avg. Score                478 non-null object
SAT Writing Avg. Score             478 non-null object
dtypes: object(6)
memory usage: 22.5+ KB


We notice that the columns are not in numeric value, let's fix this:

In [53]:
cols = [
    'SAT Math Avg. Score',
    'SAT Critical Reading Avg. Score',
    'SAT Writing Avg. Score'
]

for c in cols:
    data['sat_results'][c] = pd.to_numeric(data['sat_results'][c], errors='coerce')

In [54]:
data['sat_results'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 478 entries, 0 to 477
Data columns (total 6 columns):
DBN                                478 non-null object
SCHOOL NAME                        478 non-null object
Num of SAT Test Takers             478 non-null object
SAT Critical Reading Avg. Score    421 non-null float64
SAT Math Avg. Score                421 non-null float64
SAT Writing Avg. Score             421 non-null float64
dtypes: float64(3), object(3)
memory usage: 22.5+ KB


Lets look at the range of the scores for consistency:

In [55]:
data['sat_results'].iloc[:,3:6].max()

SAT Critical Reading Avg. Score    679.0
SAT Math Avg. Score                735.0
SAT Writing Avg. Score             682.0
dtype: float64

In [56]:
data['sat_results'].iloc[:,3:6].min()

SAT Critical Reading Avg. Score    279.0
SAT Math Avg. Score                312.0
SAT Writing Avg. Score             286.0
dtype: float64

Everything is ok, now let's create a column that totals up the SAT scores for the different sections of the exam. This will make it much easier to correlate scores with demographic factors because we'll be working with a single number, rather than three different ones.

In [57]:
data['sat_results']['sat_score'] = data['sat_results'][cols[0]] \
                                 + data['sat_results'][cols[1]] \
                                 + data['sat_results'][cols[2]]
data['sat_results'].head()

,DBN,SCHOOL NAME,Num of SAT Test Takers,SAT Critical Reading Avg. Score,SAT Math Avg. Score,SAT Writing Avg. Score,sat_score
0,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,29,355.0,404.0,363.0,1122.0
1,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,91,383.0,423.0,366.0,1172.0
2,01M450,EAST SIDE COMMUNITY SCHOOL,70,377.0,402.0,370.0,1149.0
3,01M458,FORSYTH SATELLITE ACADEMY,7,414.0,401.0,359.0,1174.0
4,01M509,MARTA VALLE HIGH SCHOOL,44,390.0,433.0,384.0,1207.0


In [58]:
data['sat_results']['DBN'].duplicated().sum()

0

## Combining in one data frame

Now we have all columns with a DBN column with unique values. Let's combiine them:

In [59]:
print(data['sat_results'].shape)
print(data['ap_2010'].shape)
print(data['graduation'].shape)

(478, 7)
(257, 5)
(405, 23)


In [60]:
combined = data['sat_results']
combined = combined.merge(data['ap_2010'], on='DBN', how='left')
combined = combined.merge(data['graduation'], on='DBN', how='left')
print(combined.shape)
combined.head()

(478, 33)


,DBN,SCHOOL NAME,Num of SAT Test Takers,SAT Critical Reading Avg. Score,SAT Math Avg. Score,SAT Writing Avg. Score,sat_score,SchoolName,AP Test Takers,Total Exams Taken,...,Regents w/o Advanced - n,Regents w/o Advanced - % of cohort,Regents w/o Advanced - % of grads,Local - n,Local - % of cohort,Local - % of grads,Still Enrolled - n,Still Enrolled - % of cohort,Dropped Out - n,Dropped Out - % of cohort
0,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,29,355.0,404.0,363.0,1122.0,NaN,NaN,NaN,...,36,46.2%,83.7%,7,9%,16.3%,16,20.5%,11,14.1%
1,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,91,383.0,423.0,366.0,1172.0,UNIVERSITY NEIGHBORHOOD H.S.,39.0,49.0,...,34,27.4%,64.2%,11,8.9%,20.8%,46,37.1%,20,16.100000000000001%
2,01M450,EAST SIDE COMMUNITY SCHOOL,70,377.0,402.0,370.0,1149.0,EAST SIDE COMMUNITY HS,19.0,21.0,...,67,74.400000000000006%,95.7%,3,3.3%,4.3%,15,16.7%,5,5.6%
3,01M458,FORSYTH SATELLITE ACADEMY,7,414.0,401.0,359.0,1174.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01M509,MARTA VALLE HIGH SCHOOL,44,390.0,433.0,384.0,1207.0,NaN,NaN,NaN,...,23,27.4%,48.9%,7,8.300000000000001%,14.9%,25,29.8%,5,6%


We still have to merge class_size, demographics, survey, and hs_directory into combined. Because these files contain information that's more valuable to our analysis and also have fewer missing DBN values, we'll use the inner join type.

In [61]:
to_merge = ['class_size', 'demographics', 'survey', 'hs_directory']

for m in to_merge:
    combined = combined.merge(data[m], on='DBN', how='inner')

print(combined.shape)
combined.head(5)


(362, 165)


,DBN,SCHOOL NAME,Num of SAT Test Takers,SAT Critical Reading Avg. Score,SAT Math Avg. Score,SAT Writing Avg. Score,sat_score,SchoolName,AP Test Takers,Total Exams Taken,...,priority10,Location 1,Community Board,Council District,Census Tract,BIN,BBL,NTA,lat,lon
0,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,29,355.0,404.0,363.0,1122.0,NaN,NaN,NaN,...,NaN,"220 Henry Street\nNew York, NY 10002\n(40.7137...",3.0,1.0,201.0,1003223.0,1.002690e+09,Lower East Side ...,40.713764,-73.985260
1,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,91,383.0,423.0,366.0,1172.0,UNIVERSITY NEIGHBORHOOD H.S.,39.0,49.0,...,NaN,"200 Monroe Street\nNew York, NY 10002\n(40.712...",3.0,1.0,202.0,1003214.0,1.002590e+09,Lower East Side ...,40.712332,-73.984797
2,01M450,EAST SIDE COMMUNITY SCHOOL,70,377.0,402.0,370.0,1149.0,EAST SIDE COMMUNITY HS,19.0,21.0,...,NaN,"420 East 12 Street\nNew York, NY 10009\n(40.72...",3.0,2.0,34.0,1005974.0,1.004390e+09,East Village ...,40.729783,-73.983041
3,01M509,MARTA VALLE HIGH SCHOOL,44,390.0,433.0,384.0,1207.0,NaN,NaN,NaN,...,NaN,"145 Stanton Street\nNew York, NY 10002\n(40.72...",3.0,1.0,3001.0,1004323.0,1.003540e+09,Chinatown ...,40.720569,-73.985673
4,01M539,"NEW EXPLORATIONS INTO SCIENCE, TECHNOLOGY AND ...",159,522.0,574.0,525.0,1621.0,"NEW EXPLORATIONS SCI,TECH,MATH",255.0,377.0,...,NaN,"111 Columbia Street\nNew York, NY 10002\n(40.7...",3.0,2.0,2201.0,1004070.0,1.003350e+09,Lower East Side ...,40.718725,-73.979426


In [62]:
combined.isnull().sum()

DBN                                       0
SCHOOL NAME                               0
Num of SAT Test Takers                    0
SAT Critical Reading Avg. Score          28
SAT Math Avg. Score                      28
SAT Writing Avg. Score                   28
sat_score                                28
SchoolName                              135
AP Test Takers                          154
Total Exams Taken                       154
Number of Exams with scores 3 4 or 5    222
Demographic                              53
School Name                              53
Cohort                                   53
Total Cohort                             53
Total Grads - n                          53
Total Grads - % of cohort                53
Total Regents - n                        53
Total Regents - % of cohort              53
Total Regents - % of grads               53
Advanced Regents - n                     53
Advanced Regents - % of cohort           53
Advanced Regents - % of grads   

## Sumary and Discussion

We had data coming from multiple files loaded in multiple dataframes needed to be combined. The 'DBN' column played a key role in the last merging steps, we assured in each previous that this column was unique in the dataframes and in particular case of 'class_size' we had to generate this columns. In the 'DBN' case, 'lat', 'long', 'sat_results' or whenever facing a situation that I need to transform data and generate something new from it I commonly do a consistency test, just to reduce the chance of propague some "dirty". Look at range of values and use regular expressions can be used for this consistency test. Many times we have numerical values in string format and we have to convert it to a numerial type. We must ensure we are agregating data that talks about the same thing or if its the case of doing some extrapolation, explicity say it and keep that in mind.  

The diagram below summarizes the process. Rectangles represents the dataframes and in diamonds the data manipulation operations. 

In [63]:
p = figure(plot_width=960, plot_height=800, x_range=(0,100), y_range=(0,100))
p.image_url(url=['./data_prep_diagram.svg'], x=0, y=100,w=100,h=100)
show(p)

We still have missing values to deal with, but this will be left for later, maybe even another notebook since some models can deal with missing values. 

The idea of this project come from an iterative mission I have done at dataquest.io, in this notebook I rearranged the structure and added some of my considerations.